# Sistemas de Recomendacoes

Recomendação de filmes para diversos usuários, de acordo com a similaridade entre os seus gostos e os gostos de outros usuários.

## Imports

In [12]:
import numpy as np
import pandas as pd
from avaliacoes import avaliacoes
from math import sqrt

## Base de dados simples

In [18]:
avaliacoes

{'Ana': {'Freddy x Jason': 2.5,
  'O Ultimato Bourne': 3.5,
  'Star Trek': 3.0,
  'Exterminador do Futuro': 3.5,
  'Norbit': 2.5,
  'Star Wars': 3.0},
 'Marcos': {'Freddy x Jason': 3.0,
  'O Ultimato Bourne': 3.5,
  'Star Trek': 1.5,
  'Exterminador do Futuro': 5.0,
  'Star Wars': 3.0,
  'Norbit': 3.5},
 'Pedro': {'Freddy x Jason': 2.5,
  'O Ultimato Bourne': 3.0,
  'Exterminador do Futuro': 3.5,
  'Star Wars': 4.0},
 'Claudia': {'O Ultimato Bourne': 3.5,
  'Star Trek': 3.0,
  'Star Wars': 4.5,
  'Exterminador do Futuro': 4.0,
  'Norbit': 2.5},
 'Adriano': {'Freddy x Jason': 3.0,
  'O Ultimato Bourne': 4.0,
  'Star Trek': 2.0,
  'Exterminador do Futuro': 3.0,
  'Star Wars': 3.0,
  'Norbit': 2.0},
 'Janaina': {'Freddy x Jason': 3.0,
  'O Ultimato Bourne': 4.0,
  'Star Wars': 3.0,
  'Exterminador do Futuro': 5.0,
  'Norbit': 3.5},
 'Leonardo': {'O Ultimato Bourne': 4.5,
  'Norbit': 1.0,
  'Exterminador do Futuro': 4.0}}

## Metodos

In [55]:
def euclidiana(base, usuario1, usuario2):
    
    si = {}
    for item in base[usuario1]:
        if item in base[usuario2]:
            si[item] = 1

    if len(si) == 0:
        return 0

    soma = sum([pow(base[usuario1][item] - base[usuario2][item], 2)
                for item in base[usuario1] if item in base[usuario2]])
    
    return 1/(1 + sqrt(soma))

def getSimilares(base, usuario):
    
    similaridade = [(euclidiana(base, usuario, outro), outro)
                    for outro in base if outro != usuario]
    similaridade.sort()
    similaridade.reverse()
    
    return similaridade[0:30]
    
def getRecomendacoesUsuario(base, usuario):
    
    totais={}
    somaSimilaridade={}
    for outro in base:
        if outro == usuario: continue
        similaridade = euclidiana(base, usuario, outro)

        if similaridade <= 0: continue

        for item in base[outro]:
            if item not in base[usuario]:
                totais.setdefault(item, 0)
                totais[item] += base[outro][item] * similaridade
                somaSimilaridade.setdefault(item, 0)
                somaSimilaridade[item] += similaridade
                
    rankings=[(round(total / somaSimilaridade[item], 2), item) for item, total in totais.items()]
    rankings.sort()
    rankings.reverse()
    
    return rankings[0:30]
                
def carregaMovieLens(path='./ml-100k'):
    
    filmes = {}
    for linha in open(path + '/u.item', encoding = "ISO-8859-1"):
        (id, titulo) = linha.split('|')[0:2]
        filmes[id] = titulo

    base = {}
    
    for linha in open(path + '/u.data', encoding = "ISO-8859-1"):
        (usuario, idfilme, nota, tempo) = linha.split('\t')
        base.setdefault(usuario, {})
        base[usuario][filmes[idfilme]] = float(nota)
        
    return base            

def calculaItensSimilares(base):
    
    result = {}
    for item in base:
        notas = getSimilares(base, item)
        result[item] = notas
        
    return result

def getRecomendacoesItens(baseUsuario, similaridadeItens, usuario):
    
    notasUsuario = baseUsuario[usuario]
    notas={}
    totalSimilaridade={}
    for (item, nota) in notasUsuario.items():
        for (similaridade, item2) in similaridadeItens[item]:
            if item2 in notasUsuario:
                continue
            
            notas.setdefault(item2, 0)
            notas[item2] += similaridade * nota
            totalSimilaridade.setdefault(item2,0)
            totalSimilaridade[item2] += similaridade
            
    rankings=[(round(score/totalSimilaridade[item], 2), item) for item, score in notas.items()]
    rankings.sort()
    rankings.reverse()
    
    return rankings

## Main

In [56]:
getRecomendacoesUsuario(avaliacoes, 'Janaina')

[(2.2, 'Star Trek')]

In [57]:
getRecomendacoesUsuario(avaliacoes, 'Pedro')

[(2.48, 'Star Trek'), (2.45, 'Norbit')]

## Main usando a base do movielens

In [61]:
# carrega a base de dados
base = carregaMovieLens()

In [62]:
getRecomendacoesUsuario(base, '100')

[(5.0, 'They Made Me a Criminal (1939)'),
 (5.0, 'Star Kid (1997)'),
 (5.0, "Someone Else's America (1995)"),
 (5.0, 'Santa with Muscles (1996)'),
 (5.0, 'Saint of Fort Washington, The (1993)'),
 (5.0, 'Prefontaine (1997)'),
 (5.0, 'Marlene Dietrich: Shadow and Light (1996) '),
 (5.0, 'Great Day in Harlem, A (1994)'),
 (5.0, 'Entertaining Angels: The Dorothy Day Story (1996)'),
 (5.0, 'Aiqing wansui (1994)'),
 (4.59, "Some Mother's Son (1996)"),
 (4.58, 'Faust (1994)'),
 (4.57, 'Pather Panchali (1955)'),
 (4.55, 'Anna (1996)'),
 (4.54, 'Close Shave, A (1995)'),
 (4.5, 'Kaspar Hauser (1993)'),
 (4.48, 'Wrong Trousers, The (1993)'),
 (4.48, 'Shawshank Redemption, The (1994)'),
 (4.48, "Schindler's List (1993)"),
 (4.48, 'Letter From Death Row, A (1998)'),
 (4.46, 'Casablanca (1942)'),
 (4.45, 'Maya Lin: A Strong Clear Vision (1994)'),
 (4.45, 'Everest (1998)'),
 (4.44, 'Wallace & Gromit: The Best of Aardman Animation (1996)'),
 (4.42, 'Perfect Candidate, A (1996)'),
 (4.39, 'Rear Window 

In [63]:
getRecomendacoesUsuario(base, '200')

[(5.0, 'They Made Me a Criminal (1939)'),
 (5.0, 'Star Kid (1997)'),
 (5.0, "Someone Else's America (1995)"),
 (5.0, 'Santa with Muscles (1996)'),
 (5.0, 'Saint of Fort Washington, The (1993)'),
 (5.0, 'Prefontaine (1997)'),
 (5.0, 'Marlene Dietrich: Shadow and Light (1996) '),
 (5.0, 'Great Day in Harlem, A (1994)'),
 (5.0, 'Entertaining Angels: The Dorothy Day Story (1996)'),
 (5.0, 'Aiqing wansui (1994)'),
 (4.75, 'Pather Panchali (1955)'),
 (4.64, 'Maya Lin: A Strong Clear Vision (1994)'),
 (4.63, "Some Mother's Son (1996)"),
 (4.63, 'Anna (1996)'),
 (4.57, 'Letter From Death Row, A (1998)'),
 (4.52, 'Lamerica (1994)'),
 (4.51, 'Close Shave, A (1995)'),
 (4.48, 'Shawshank Redemption, The (1994)'),
 (4.46, 'Third Man, The (1949)'),
 (4.43, 'Wallace & Gromit: The Best of Aardman Animation (1996)'),
 (4.42, 'Usual Suspects, The (1995)'),
 (4.41, 'Rear Window (1954)'),
 (4.41, 'Faust (1994)'),
 (4.41, 'Everest (1998)'),
 (4.4, 'World of Apu, The (Apur Sansar) (1959)'),
 (4.39, 'Bitter 